# Scraper
### Scrape Toronto Postal Codes



In [6]:
import pandas as pd
import numpy as np

#Visualization Libraries
import folium
import geopy

#import Beatiful Soup for allowing easier parsing of html
from bs4 import BeautifulSoup

#http request handler
import urllib.request as req

In [33]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = req.urlopen(url)

#Invoke beautiful Soup to parse out the page
resulting_page = BeautifulSoup(page, "html.parser")
resulting_page.prettify()

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   List of postal codes of Canada: M - Wikipedia\n  </title>\n  <script>\n   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"3f996570-256b-4f45-92a3-87ba6c847372","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canad

In [34]:
table=resulting_page.find('table', class_='wikitable sortable')
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [39]:
# Init arrays for cell data
A=[]
B=[]
C=[]

#iterate for cell data
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        if "Not assigned\n" in cells[1]:
            continue
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [44]:
# Convert above to Pandas df
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

# Clean \n's out of data
df = df.replace('\n','', regex=True)

# Eliminate unnecessary extra rows
df.groupby('Postal Code').agg({'Neighborhood': ', '.join, 
                               'Borough':'first' }).reset_index()

df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [54]:
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [82]:
import geocoder # import geocoder
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="ny_explorer")

# initialize your variable to None
codes = df['Borough']

Lat = []
Long = []

for code in codes:
    strCode = '{}, Toronto, Ontario'.format(code)
    try:
        loc = locator.geocode( strCode )
        print('loc: ', loc)
        lat = loc.latitude
        longi = loc.longitude
        Lat.append(lat)
        Long.append(longi) 
    except:
        pass

df['Latitude'] = Lat
df['Longitude'] = Long

df.head()

loc:  North York, Toronto, Golden Horseshoe, Ontario, Canada
loc:  North York, Toronto, Golden Horseshoe, Ontario, Canada
loc:  Downtown Yonge, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 2C3, Canada
loc:  North York, Toronto, Golden Horseshoe, Ontario, Canada
loc:  Downtown Yonge, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 2C3, Canada
loc:  Etobicoke, Etobicoke Centre, Etobicoke, Toronto, Golden Horseshoe, Ontario, M9C 0B1, Canada
loc:  Scarborough, Scarborough Centre, Scarborough, Toronto, Golden Horseshoe, Ontario, M1P 4N7, Canada
loc:  North York, Toronto, Golden Horseshoe, Ontario, Canada
loc:  East York, Toronto, Golden Horseshoe, Ontario, Canada
loc:  Downtown Yonge, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 2C3, Canada
loc:  North York, Toronto, Golden Horseshoe, Ontario, Canada
loc:  Etobicoke, Etobicoke Centre, Etobicoke, Toronto, Golden Horseshoe, Ontario, M9C 0B1, Canada
loc:  Scarborough, Scarbor

loc:  Scarborough, Scarborough Centre, Scarborough, Toronto, Golden Horseshoe, Ontario, M1P 4N7, Canada
loc:  Ontario Travel Information Centre, Front Street West, Financial District, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5J 1E3, Canada
loc:  Downtown Yonge, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 2C3, Canada
loc:  Etobicoke, Etobicoke Centre, Etobicoke, Toronto, Golden Horseshoe, Ontario, M9C 0B1, Canada
loc:  Etobicoke, Etobicoke Centre, Etobicoke, Toronto, Golden Horseshoe, Ontario, M9C 0B1, Canada
loc:  Scarborough, Scarborough Centre, Scarborough, Toronto, Golden Horseshoe, Ontario, M1P 4N7, Canada
loc:  Downtown Yonge, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 2C3, Canada
loc:  Downtown Yonge, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 2C3, Canada
loc:  Etobicoke, Etobicoke Centre, Etobicoke, Toronto, Golden Horseshoe, Ontario, M9C 0B1, Canada
loc:  Etobicoke, Etobicok

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.754326,-79.449117
1,M4A,North York,Victoria Village,43.754326,-79.449117
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.656322,-79.380916
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.754326,-79.449117
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.656322,-79.380916


In [77]:
location = locator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Postal Code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#aa35cc',
        fill_opacity=0.7,
        parse_html=False).add_to( map_toronto )   
    
map_toronto

In [78]:
df.shape

(103, 5)